In [19]:
import pandas as pd
import numpy as np
import sklearn
from time import time

data_offline = pd.read_csv('ccf_offline.csv')
data_online = pd.read_csv('ccf_online.csv')
    
def sent_coupon(value):
    if value != 'null':
        return 1
    else: 
        return 0
    
def used_coupon(a,b):
    if a == 1 and b != 'null':
        return 1
    else:
        return 0

In [20]:
# Online

def get_custMerch(data):
    data['Coupon Received'] = data['Coupon_id'].map(sent_coupon)
    data['Coupon Used'] = data.apply(lambda x: used_coupon(x['Coupon Received'], x['Date']), axis = 1)
    grouped = data.groupby(['User_id', 'Merchant_id'], as_index = False)[['Coupon Received','Coupon Used']].agg(sum)
    grouped["Customer's Total Usage Fraction for Merchant"] = grouped['Coupon Used'] / grouped['Coupon Received'].fillna(0)
    grouped["Customer's Total Usage for Merchant's Coupons"] = grouped["Coupon Used"]
    grouped["Customer's Total Received by Merchant"] = grouped["Coupon Received"]
    del grouped["Coupon Used"]
    del grouped["Coupon Received"]
    merge0 = grouped.merge(data, on = ['User_id','Merchant_id'] )
    receive_frac = data.groupby(['User_id', 'Merchant_id'])[['Coupon Received']].agg(sum)/data.groupby(['User_id'])[['Coupon Received']].agg(sum)
    receive_frac = receive_frac.reset_index()
    merged1 = receive_frac.merge(merge0, on = ['User_id','Merchant_id'])
    merged1["Customer's Inbox Share by Merchant"] = merged1['Coupon Received_x'].fillna(0)
    del merged1['Coupon Received_x']
    merged1['Coupon Received'] = merged1['Coupon Received_y']
    del merged1['Coupon Received_y']
    
    # Note: this next feature is really more merchant-specific
    merchant_group = data.groupby(['Merchant_id', 'User_id'])[['Coupon Received', 'Coupon Used']].agg(sum)
    by_merchant = merchant_group.groupby(['Merchant_id'])[['Coupon Received', 'Coupon Used']].agg(sum)
    by_merchant["Merchant's Total Usage Fraction"] = by_merchant['Coupon Used'] / by_merchant['Coupon Received']
    by_merchant["Merchant's Total Usage Fraction"] = by_merchant["Merchant's Total Usage Fraction"].fillna(0)
    del by_merchant['Coupon Received'], by_merchant['Coupon Used']
    by_merchant = by_merchant.reset_index()
    finalMerge= merged1.merge(by_merchant, on = ['Merchant_id'])
    
    return finalMerge


In [22]:
# Online and offline separately

# online_custMerch = get_custMerch(data_online)
# offline_custMerch = get_custMerch(data_offline)

In [23]:
# Combining online and offline

def main(data_online, data_offline):
    online_custMerch = get_custMerch(data_online)
    offline_custMerch = get_custMerch(data_offline)
    both = [online_custMerch, offline_custMerch]
    combined_df = pd.concat(both)
    return combined_df

In [24]:
combined_df = main(data_online, data_offline)
combined_df[:10]

,Action,Coupon Received,Coupon Used,Coupon_id,Customer's Inbox Share by Merchant,Customer's Total Received by Merchant,Customer's Total Usage Fraction for Merchant,Customer's Total Usage for Merchant's Coupons,Date,Date_received,Discount_rate,Distance,Merchant's Total Usage Fraction,Merchant_id,User_id
0,2.0,1,0,100145044,1.000000,1,0.0,0,null,20160331,100:10,NaN,0.116857,25104,4
1,2.0,1,0,100120354,1.000000,1,0.0,0,null,20160613,150:50,NaN,0.116857,25104,2707
2,2.0,1,0,100120354,0.047619,1,0.0,0,null,20160609,150:50,NaN,0.116857,25104,4381
3,2.0,1,0,100120354,1.000000,1,0.0,0,null,20160602,150:50,NaN,0.116857,25104,5125
4,2.0,1,0,100189568,0.500000,2,0.0,0,null,20160330,50:10,NaN,0.116857,25104,5146
5,2.0,1,0,100185708,0.500000,2,0.0,0,null,20160428,20:10,NaN,0.116857,25104,5146
6,2.0,1,0,100120354,1.000000,1,0.0,0,null,20160525,150:50,NaN,0.116857,25104,5176
7,2.0,1,0,100177820,1.000000,1,0.0,0,null,20160507,20:10,NaN,0.116857,25104,5422
8,2.0,1,0,100120354,1.000000,1,0.0,0,null,20160526,150:50,NaN,0.116857,25104,5462
9,2.0,1,0,100120354,0.333333,1,0.0,0,null,20160612,150:50,NaN,0.116857,25104,5681
